In [15]:
import sympy as sym 
import numpy as np
import control as ct
from sympy import symbols, Matrix, cos, sin
from sympy.physics.mechanics import dynamicsymbols
from holonomic_sys import Holonomic_System
import control as ct

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Automated Lagrangian Workflow for a Triple Pendulum

This notebook provides an example of the structured and streamline approach I developed for deriving the dynamics and control matrices, A and B respectively, for holonomic systems. This was motivated by a larger project of mine: controlling the inverted n-pendulum attached to a driven cart. I found deriving the equations of motion for this system, by hand, intractable for anything over two pendulum bobs. By the time I created a system for deriving and linearizing the equations of motion of the inverted n-pendulum, I realized that it could be used for any holonomic system. 

This notebook structure is as follows:
1. **Defining the Inverted Three Pendulum System**: Establishing generalized coordinates, constraints, and forces.
2. **Deriving the Equations of Motion**: Initializing the 3_pendulum object and computationally deriving the equations of motion
3. **Obtaining A and B** 

## 1. System Definition

We consider the inverted 3-pendulum attached to a cart constrained to move along the x-axis (shown below). The pendulums are confined to rotate in the x-z plane and the constraints are assumed to perform no work. 

### Generalized Coordinates
- $\theta_1$: Angle of the first pendulum.
- $\theta_2$: Angle of the second pendulum.
- $\theta_3$: Angle of the third pendulum.
- x: displacement of the cart from the origin (along the x axis) 

This system has 4 degrees of freedom, and the conversion of the generalized coordinates to cartesian coordinates is given by the following function, phi. (ie phi: generalized_coordinates -> cartesian coordinates )

![Inverted Three Pendulum](3_pend.jpeg)

In [18]:
#defining masses, lengths and constraints
n = 3

m = Matrix(symbols(f"m0:{n+1}")) 
l = Matrix(symbols(f"l1:{n+1}"))
q = Matrix(
    [dynamicsymbols("x")] + dynamicsymbols(f"theta1:{n+1}")
)


x = dynamicsymbols(f"x0:{n+1}")
y = dynamicsymbols(f"y0:{n+1}")
z = dynamicsymbols(f"z0:{n+1}")
cart_coor = Matrix([*x,*y,*z])

def phi(q):
    x= [q[0]]
    y= [0]
    z= [0]
    for i in range(1,n+1):
        x.append(x[i-1]+ l[i-1]*sin(q[i]))
        y.append(0) #encodes that the system is confined to the x-z plane
        z.append(z[i-1]+ l[i-1]*cos(q[i]))
        

    return sym.Matrix(x+y+z)

#provides the conversion from (x, theta1, theta2, theta3) -> cartesian coordinates (with the structure x's, y's, and then z's)
#this implicitly defines the constraints on the system
phi(q) 

Matrix([
[                                                            x(t)],
[                                        l1*sin(theta1(t)) + x(t)],
[                    l1*sin(theta1(t)) + l2*sin(theta2(t)) + x(t)],
[l1*sin(theta1(t)) + l2*sin(theta2(t)) + l3*sin(theta3(t)) + x(t)],
[                                                               0],
[                                                               0],
[                                                               0],
[                                                               0],
[                                                               0],
[                                               l1*cos(theta1(t))],
[                           l1*cos(theta1(t)) + l2*cos(theta2(t))],
[       l1*cos(theta1(t)) + l2*cos(theta2(t)) + l3*cos(theta3(t))]])

In [19]:
#gravitational and driving forces

u = Matrix([dynamicsymbols("u")]) #control input
g = symbols("g") #gravitational constant

f_cart = sym.Matrix([
    [0, 0,-m[i]*g] for i in range(1,n+1)
]).row_insert(0, Matrix([u[0],0,0]).T) #encodes the forcing term on the cart
#it is not necessary to include the gravitational force on the cart, since it is confined to z = 0

u = Matrix([u])

f_cart #note that the ith row corresponds to the force on the ith point mass

Matrix([
[u(t), 0,     0],
[   0, 0, -g*m1],
[   0, 0, -g*m2],
[   0, 0, -g*m3]])

## 2. Deriving the Equations of Motion



In [20]:
inv_three_pendulum = Holonomic_System(
    f_cart,
    m,
    q, 
    cart_coor,
    u,
    phi
) 

In [28]:
#Exemplifies why you would certainly not want to do this by hand!
inv_three_pendulum.eqs_motion

Matrix([[1.0*m0*Derivative(x(t), (t, 2)) + 1.0*m1*(-l1*sin(theta1(t))*Derivative(theta1(t), t)**2 + l1*cos(theta1(t))*Derivative(theta1(t), (t, 2)) + Derivative(x(t), (t, 2))) + 1.0*m2*(-l1*sin(theta1(t))*Derivative(theta1(t), t)**2 + l1*cos(theta1(t))*Derivative(theta1(t), (t, 2)) - l2*sin(theta2(t))*Derivative(theta2(t), t)**2 + l2*cos(theta2(t))*Derivative(theta2(t), (t, 2)) + Derivative(x(t), (t, 2))) + 1.0*m3*(-l1*sin(theta1(t))*Derivative(theta1(t), t)**2 + l1*cos(theta1(t))*Derivative(theta1(t), (t, 2)) - l2*sin(theta2(t))*Derivative(theta2(t), t)**2 + l2*cos(theta2(t))*Derivative(theta2(t), (t, 2)) - l3*sin(theta3(t))*Derivative(theta3(t), t)**2 + l3*cos(theta3(t))*Derivative(theta3(t), (t, 2)) + Derivative(x(t), (t, 2))) - u(t), 1.0*l1*(-g*m1*sin(theta1(t)) - g*m2*sin(theta1(t)) - g*m3*sin(theta1(t)) + l1*m1*Derivative(theta1(t), (t, 2)) + l1*m2*Derivative(theta1(t), (t, 2)) + l1*m3*Derivative(theta1(t), (t, 2)) + l2*m2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2 + 

## 3. Obtaining A and B

From the definition of the contraints, the generalized coordinates corresponding to the vertical position, as shown below, are $\theta{}_i = 0$, $\forall i$ and $x=$ anything. To obtain the dynamics and control matrices, call control_init, and supply a dictionary which maps each generalized coordinate to the values which the system is to be linearized about. 

In [22]:
state0 = dict(zip(q, sym.zeros(n+1, 1)))
A,B = inv_three_pendulum.control_init(state0)

In [23]:
A

Matrix([
[0,                                             0,                                        0,                       0, 1, 0, 0, 0],
[0,                                             0,                                        0,                       0, 0, 1, 0, 0],
[0,                                             0,                                        0,                       0, 0, 0, 1, 0],
[0,                                             0,                                        0,                       0, 0, 0, 0, 1],
[0,                  1.0*(-g*m1 - g*m2 - g*m3)/m0,                                        0,                       0, 0, 0, 0, 0],
[0, 1.0*(m0 + m1)*(g*m1 + g*m2 + g*m3)/(l1*m0*m1),               1.0*(-g*m2 - g*m3)/(l1*m1),                       0, 0, 0, 0, 0],
[0,             1.0*(-g*m1 - g*m2 - g*m3)/(l2*m1), -1.0*(m1 + m2)*(-g*m2 - g*m3)/(l2*m1*m2),       -1.0*g*m3/(l2*m2), 0, 0, 0, 0],
[0,                                             0,               1.0*(-g*m

In [24]:
B

Matrix([
[           0],
[           0],
[           0],
[           0],
[      1.0/m0],
[-1.0/(l1*m0)],
[           0],
[           0]])

In [25]:
#testing whether the system is controllable

parameters = list(m)+list(l) + [g]#masses and lengths of the pendulums
sample_values = list(sym.ones(len(parameters)-1))+[9.8] #values chosen for simplicity

parameters_to_sample_values = dict(zip(parameters, sample_values)) 

A_numerical = np.array(A.subs(parameters_to_sample_values), dtype= np.float32)
B_numerical = np.array(B.subs(parameters_to_sample_values), dtype= np.float32)

In [26]:
#recall that the system is controllable if the controllability matrix has 
# rank = dimension of dynamics matrix

np.linalg.matrix_rank(ct.ctrb(A_numerical,B_numerical))

np.int64(8)

## Work Cited

Refer to "Schaum's Outlines: Lagrangian Mechanics" (Wells) for a more in depth coverage of variational mechanics.